### **Importing Libraries**

In [1]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import GameState
from Game_and_Player import *
import time

### **Helper Functions**

In [2]:
class MCTSNode:
    def __init__(self, game_state, parent=None,move_taken=None,owner=None):
        self.game_state = game_state  # The game state at tho;his node
        self.parent = parent  # Parent node
        self.children = []  # List of child nodes
        self.visits = 0  # Number of times this node has been visited
        self.total_reward = 0  # Total reward (win/loss)
        self.untried_moves = self.game_state.get_legal_moves()  # Legal moves from this state
        self.move_taken = move_taken # move that led to this game state
        self.visited_nodes = {}
        self.owner=owner

    def expand(self,move):
          game_state = self.game_state
          new_game_state = copy.deepcopy(game_state)  # Copy external game state, not self.game_state
          self.untried_moves.remove(move)
          # Store current player before applying move (they took the action)
          node_owner = self.game_state.players[self.game_state.current_player_index].name
          child_game_state = new_game_state.apply_move(move)  # Apply move without modifying original

          child_node = MCTSNode(child_game_state, parent=self, move_taken=move,owner= node_owner)
          self.children.append(child_node)

          return child_node

    # def simulate(self,game_state):

    #     game_state_new = copy.deepcopy(game_state)
    #     starting_player_index = game_state.current_player_index
    #     starting_player_name = game_state.players[starting_player_index].name

    #     while not game_state_new.is_terminal():
    #         legal_moves = game_state_new.get_legal_moves()
    #         if not legal_moves:
    #             break
    #         move = random.choice(legal_moves)
    #         game_state_new.apply_move(move)

    #     rewards=game_state_new.tricks_won

    #     # winner_name = max(rewards, key=rewards.get)

    #     # return game_state_new.tricks_won[starting_player_name]
    #     max_tricks = max(rewards.values(), default=1)  # Avoid division by zero
    #     if max_tricks == 0:
    #         max_tricks = 1  # Avoid division by zero
    #     reward = (game_state.players[starting_player_index].tricks_won - max_tricks) / max_tricks
    #     return reward

    def simulate(self, game_state):
          """
          Simulates a random rollout and returns a reward for each player.
          """
          game_state_new = copy.deepcopy(game_state)

          while not game_state_new.is_terminal():
              legal_moves = game_state_new.get_legal_moves()
              if not legal_moves:
                  break
              move = random.choice(legal_moves)
              game_state_new.apply_move(move)

          # Assign rewards based on tricks won
          rewards = {}
          max_tricks = max(game_state_new.tricks_won.values(), default=1)
          if max_tricks == 0:
              max_tricks = 1

          for player in game_state_new.players:
              player_tricks = game_state_new.tricks_won[player.name]
              rewards[player.name] = (player_tricks - max_tricks) / max_tricks  # Normalize reward
              # rewards[player.name] = player_tricks

          return rewards  # Now returns a separate reward for each player






    # def backpropagate(self, reward):
    #     self.visits += 1
    #     self.total_reward += reward
    #     self.visited_nodes[id(self)] = (self.move_taken, self.visits)
    #     if self.parent:
    #         self.parent.backpropagate(reward)

    def backpropagate(self, rewards):
          """
          Backpropagates rewards up the tree, giving rewards only to the player who took the move at each node.

          Args:
              rewards (dict): A dictionary where rewards[player_name] contains the reward for that player.
          """
          self.visits += 1

          if self.owner:  # Root node has no owner
              reward = rewards.get(self.owner, 0)  # Get only the reward for this node's owner
              self.total_reward += reward

          # Continue backpropagation up the tree
          if self.parent:
              self.parent.backpropagate(rewards)


    def ucb1(self,child, c=0.707):
        parent_visits = child.parent.visits if child.parent else 1

        if child.visits == 0:
            return float("inf")  # Prioritize unvisited nodes
        return (child.total_reward / child.visits) + c * math.sqrt(2*math.log(parent_visits) / child.visits)


    def best_child(self, c=4):
        node = self
        best = max(node.children, key=lambda child: node.ucb1(child, c))
        return best

    # def tree_policy(self, c=1.4):
    #     """
    #     Traverses the tree from the current node to a leaf node.
    #     Expands the node if there are untried moves.
    #     """

    #     current_node = self
    #     while not current_node.game_state.is_terminal():

    #         if current_node.untried_moves:
    #             return current_node.expand(random.choice(current_node.untried_moves))
    #         else:

    #             current_node = current_node.best_child( c)
    #     return current_node

    def get_all_nodes(self):
        """Recursively return a list of all nodes in the subtree starting from this node."""
        nodes = [self]
        for child in self.children:
            nodes.extend(child.get_all_nodes())
        return nodes










In [3]:

def mcts_search(root_node, num_simulations=100):

    game_state_new = root_node.game_state
    game_state_copy1 = copy.deepcopy(game_state_new)
    root = MCTSNode(game_state_copy1)  # Root node of the MCTS tree

    # root = root_node

    for _ in range(num_simulations):
        node = root
        game_state_copy = copy.deepcopy(root.game_state)

        # Tree Traversal
        while node.untried_moves == [] and node.children:
            node = node.best_child(c=0.707)
            # node = node.expand( move)
            game_state_copy.apply_move(node.move_taken)

        # Expansion
        if node.untried_moves:
        # while node.untried_moves:
            move = random.choice(node.untried_moves)
            node = node.expand( move)
            game_state_copy.apply_move(move)

        # Simulation
        reward = node.simulate(game_state_copy)

        # Backpropagation
        node.backpropagate(reward)


    best_child = max(root.children, key=lambda child: child.visits)
    # best_child = max(root.children, key=lambda child: child.total_reward / (child.visits + 1e-6))

    # all_nodes = root.get_all_nodes()
    # for node in all_nodes:
    #     print(f"Move: {node.move_taken}, Visits: {node.visits}, Total Reward: {node.total_reward}")

    return best_child.move_taken


## **Random Play Vs Monte Carlo**

In [4]:
def evaluate_monte_carlo(num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)


        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:  # Monte Carlo player is Player 0
                root_node = MCTSNode(game_state)
                best_move = mcts_search(root_node, num_simulations)
            else:
                legal_moves1 = game_state.get_legal_moves()
                # print(game_state.players[game_state.current_player_index].hand)


                if not legal_moves1:
                    print(f"No legal moves for player {game_state.current_player_index}. Ending game.")
                    break

                best_move = random.choice(legal_moves1)

            game_state.apply_move(best_move)


        rewards = game_state.tricks_won
        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            print(f"Monte Carlo player did not win: {winner_name} ")

    # Calculate and print win rate and standard deviation
    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



In [5]:
start=time.time()
evaluate_monte_carlo(num_games=1000, num_simulations=100)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo player did not win: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Alice 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Alice 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Alice 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Bob 
Monte Carlo player did not win: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player did not win: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player won: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Alice 
Monte Carlo player won: Charlie 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Alice 
Monte Carlo player did not win: Alice 


## **Rule Based Play Vs Monte Carlo**

In [8]:
def rule_based_move(game_state):
    legal_moves = game_state.get_legal_moves()
    return max(legal_moves, key=lambda card: card.rank)


def evaluate_monte_carlo(num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)


        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:  # Monte Carlo player is Player 0
                root_node = MCTSNode(game_state)
                best_move = mcts_search(root_node, num_simulations)
            else:
                legal_moves = game_state.get_legal_moves()
                best_move = rule_based_move(game_state)

                if not legal_moves:
                    print(f"No legal moves for player {game_state.current_player_index}. Ending game.")
                    break



            game_state.apply_move(best_move)


        rewards = game_state.tricks_won
        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            print(f"Monte Carlo player did not win: {winner_name} ")

    # Calculate and print win rate and standard deviation
    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Rule Based Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")




In [10]:
start=time.time()
evaluate_monte_carlo(num_games=1000, num_simulations=100)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo player won: Charlie 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Charlie 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Alice 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Alice 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player did not win: Charlie 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Charlie 
Monte Carlo player won: Charlie 
Monte Carlo player did not win: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Bob 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Alice 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Charlie 
Monte Carlo player won: Bob 
Monte Carlo player won: Alice 
Monte Carlo player won: Alice 
Monte Carlo player